In [1]:
!nvidia-smi

Mon Jul  1 16:39:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [23]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet transformers datasets[audio]
!pip install gradio

In [21]:
import torch
from transformers import MusicgenForConditionalGeneration, AutoProcessor
from IPython.display import Audio
import scipy.io.wavfile
import gradio as gr
import numpy as np

In [4]:
print("Loading model and processor...")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
print("Model and processor loaded successfully.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Using device: {device}")

MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): L

In [26]:
def generate_music(prompt, duration, randomness):
    try:
        inputs = processor(
            text=[prompt],
            padding=True,
            return_tensors="pt",
        )

        audio_values = model.generate(
            **inputs.to(device),
            do_sample=True,
            guidance_scale=3,
            temperature=randomness,
            max_new_tokens=int(duration * model.config.audio_encoder.frame_rate)
        )

        audio = audio_values[0, 0].cpu().numpy()

        output_file = "generated_music.wav"
        scipy.io.wavfile.write(output_file, rate=model.config.audio_encoder.sampling_rate, data=audio)

        return output_file
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [27]:
def gradio_generate_music(prompt, duration, randomness):
    result = generate_music(prompt, duration, randomness)
    if result:
        return result
    else:
        return None

iface = gr.Interface(
    fn=gradio_generate_music,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="Enter a description of the music you want..."),
        gr.Slider(minimum=5, maximum=30, value=10, step=5, label="Duration (seconds)"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.8, step=0.1, label="Randomness"),
    ],
    outputs=gr.Audio(label="Generated Music"),
    title="MusicGen: AI Music Generator",
    description="Generate music based on text descriptions using the MusicGen AI model.",
    examples=[
        ["Upbeat pop song with catchy melody", 15, 0.8],
        ["Soft piano ballad with strings", 20, 0.7],
        ["Energetic rock anthem with guitar solo", 25, 0.9],
    ],
)

In [28]:
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b4d0e63ec44665a9e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
